# CBF (Contents Based Filtering)

In [1]:
pwd

'C:\\Users\\Jay\\Desktop\\project\\kyobobook_recommendation'

In [2]:
# module & library 

import pandas as pd

## 데이터 프레임 전처리

In [4]:
df_best = pd.read_excel('kyobo_best_all.xlsx')
df_best.head()

,순위,바코드,도서명,저자명,출판사명,출간일,정가,카테고리,담당자
0,101,9791190641159,첫 번째 디자인 케이크,지은혜(아이라이크케이크),책밥,20200820,16000,요리,1
1,79,9788982814471,연금술사,파울로 코엘료,문학동네,20181205,12000,소설,1
2,138,9791189727451,기계설비 표준품셈(2022)(CD1장포함)(양장본 HardCover),건설연구원,건설연구원,20220114,34000,기술/공학,1
3,115,9791167900791,그때 그 마음(2022 제67회 현대문학상 수상소설집),정소현 외,현대문학,20211210,15000,소설,1
4,89,9791165343842,대한민국 금기 깨기,김동연,쌤앤파커스,20210728,17000,정치사회,1


In [3]:
df_keyword = pd.read_excel('book_keyword.xlsx')
df_keyword.head()

,title,keyword,score,total
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5
3,대한민국 금기 깨기,부총리 전어 기회 반란 디지털 경제 국민,9.4,37
4,하루 한 장 아이패드 드로잉,에세이 그림 인스타그램 그리기 도안 일러스트,9.9,17


In [24]:
# category 추가 
df_CBF = df_keyword.join(df_best.set_index('도서명')['카테고리'], on='title')
df_CBF.rename(columns={'카테고리':'category'}, inplace=True)

# 카테고리와 장르 합친 새로운 컬럼 생성
df_CBF["keyword2"] = df_CBF['category'] + " " + df_CBF["keyword"]
# 중복데이터 제거
df_CBF =df_CBF.drop_duplicates(subset="title", keep='first', inplace=False, ignore_index=False)
df_CBF.reset_index(drop=True, inplace=True)
df_CBF.info()
df_CBF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     3096 non-null   object 
 1   keyword   3096 non-null   object 
 2   score     3096 non-null   float64
 3   total     3096 non-null   int64  
 4   category  3096 non-null   object 
 5   keyword2  3096 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 145.2+ KB


,title,keyword,score,total,category,keyword2
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,요리,요리 디저트 초코 아이싱 플라워 제과제빵 밀크티
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350,소설,소설 마음 모험 연금술 만물 성장소설 영혼
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5,소설,소설 순정 혜성 초파리 심사 폐허 허공
3,대한민국 금기 깨기,부총리 전어 기회 반란 디지털 경제 국민,9.4,37,정치사회,정치사회 부총리 전어 기회 반란 디지털 경제 국민
4,하루 한 장 아이패드 드로잉,에세이 그림 인스타그램 그리기 도안 일러스트,9.9,17,예술대중문화,예술대중문화 에세이 그림 인스타그램 그리기 도안 일러스트


## 함수 설정

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
# 가중평점 반영 안한 추천시스템
def find_sim_book_ver1(df_CBF, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_book = df_CBF[df_CBF['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_book.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)    
    # 2차원 데이터를 1차원으로 변환
    similar_indexes = similar_indexes.reshape(-1)
    
    return df_CBF.iloc[similar_indexes]

In [58]:
# 가중평점 반영한 추천시스템
def find_sim_book_ver2(df_CBF, sorted_ind, title_name, top_n=10):
    title_book = df_CBF[df_CBF['title'] == title_name]
    title_index = title_book.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*1)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 서적 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df_CBF.iloc[similar_indexes].sort_values('weighted_score', ascending=False)[:top_n]

In [31]:
# CountVectorizer로 학습시켰더니 3096개의 책에 대한 22882개의 키워드의 "키워드 매트릭스"가 생성되었다.

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2 1단어, 2단어까지
genre_mat = count_vect.fit_transform(df_CBF['keyword']) # 키워드 기반학습

In [32]:
print(genre_mat.shape)
print(genre_mat)

(3096, 22882)
  (0, 4727)	1
  (0, 18779)	1
  (0, 12101)	1
  (0, 20950)	1
  (0, 17168)	1
  (0, 6916)	1
  (0, 4734)	1
  (0, 18780)	1
  (0, 12103)	1
  (0, 20953)	1
  (0, 17169)	1
  (1, 5428)	1
  (1, 6123)	1
  (1, 13218)	1
  (1, 5558)	1
  (1, 9827)	1
  (1, 13591)	1
  (1, 5438)	1
  (1, 6137)	1
  (1, 13219)	1
  (1, 5559)	1
  (1, 9831)	1
  (2, 10633)	1
  (2, 22370)	1
  (2, 18790)	1
  :	:
  (3094, 6026)	1
  (3094, 3828)	1
  (3094, 6027)	1
  (3094, 8283)	1
  (3094, 11668)	1
  (3094, 1399)	1
  (3094, 22729)	1
  (3094, 8284)	1
  (3094, 3830)	1
  (3094, 11669)	1
  (3094, 1401)	1
  (3094, 20529)	1
  (3094, 6558)	1
  (3094, 22730)	1
  (3095, 16886)	1
  (3095, 1200)	1
  (3095, 1558)	1
  (3095, 10306)	1
  (3095, 20997)	1
  (3095, 13532)	1
  (3095, 16887)	1
  (3095, 1224)	1
  (3095, 13538)	1
  (3095, 10307)	1
  (3095, 1561)	1


In [33]:
# CountVectorizer로 학습시켰더니 3096개의 책에 대한 24447개의 키워드의 "키워드 매트릭스"가 생성되었다.

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))  # min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2 1단어, 2단어까지
genre_mat2 = count_vect.fit_transform(df_CBF['keyword2']) # 키워드 기반학습

In [34]:
print(genre_mat2.shape)
print(genre_mat2)

(3096, 24447)
  (0, 14852)	1
  (0, 5021)	1
  (0, 20140)	1
  (0, 12603)	1
  (0, 22462)	1
  (0, 18274)	1
  (0, 7257)	1
  (0, 14863)	1
  (0, 5028)	1
  (0, 20141)	1
  (0, 12605)	1
  (0, 22465)	1
  (0, 18275)	1
  (1, 10499)	1
  (1, 5722)	1
  (1, 6464)	1
  (1, 13912)	1
  (1, 5852)	1
  (1, 10231)	1
  (1, 14285)	1
  (1, 10519)	1
  (1, 5732)	1
  (1, 6478)	1
  (1, 13913)	1
  (1, 5853)	1
  :	:
  (3094, 8687)	1
  (3094, 12170)	1
  (3094, 1619)	1
  (3094, 24294)	1
  (3094, 20377)	1
  (3094, 8688)	1
  (3094, 4124)	1
  (3094, 12171)	1
  (3094, 1621)	1
  (3094, 22041)	1
  (3094, 6899)	1
  (3094, 24295)	1
  (3095, 14750)	1
  (3095, 17900)	1
  (3095, 14794)	1
  (3095, 1346)	1
  (3095, 1851)	1
  (3095, 10737)	1
  (3095, 22509)	1
  (3095, 14226)	1
  (3095, 17901)	1
  (3095, 1370)	1
  (3095, 14232)	1
  (3095, 10738)	1
  (3095, 1854)	1


In [35]:
# 코사인 유사도에 의해 3096개 리스트에 대한 유사한 책 계산
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim2 = cosine_similarity(genre_mat2, genre_mat2)

print(genre_sim.shape) # 키워드 기반
print(genre_sim[:10])

print(genre_sim2.shape) # 키워드2(카테고리+키워드) 기반
print(genre_sim2[:10])

(3096, 3096)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3096, 3096)
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.07692308 ... 0.12725695 0.         0.        ]
 [0.         0.07692308 1.         ... 0.12725695 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.07161149]]


In [36]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 책을 앞에서부터 순서대로 보여줌

# 0번째 책의 경우 유사도 순서 : 0번, 3494번, 813번, ..., 2401 순서
# 키워드 기반
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # ::-1 : 역순으로 정렬
print(genre_sim_sorted_ind[:5])

#키워드2 기반
genre_sim_sorted_ind2 = genre_sim2.argsort()[:, ::-1] # ::-1 : 역순으로 정렬
print(genre_sim_sorted_ind[:5])

[[   0  791 2615 ... 2055 2054 1547]
 [   1  653 2165 ... 2049 2048    0]
 [   2 2256 1088 ... 2058 2057    0]
 [   3 2875  263 ... 2054 2053    0]
 [   4 2220 1422 ... 2043 2042    0]]
[[   0  791 2615 ... 2055 2054 1547]
 [   1  653 2165 ... 2049 2048    0]
 [   2 2256 1088 ... 2058 2057    0]
 [   3 2875  263 ... 2054 2053    0]
 [   4 2220 1422 ... 2043 2042    0]]


In [38]:
# 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
title_book = df_CBF[df_CBF['title'] == '스포츠심리학의 정석(개정판)']
title_book

,title,keyword,score,total,category,keyword2
3094,스포츠심리학의 정석(개정판),불안 모범 답안 실습 과제 평가 문항 효능감 복습,10.0,2,취미실용스포츠,취미실용스포츠 불안 모범 답안 실습 과제 평가 문항 효능감 복습


In [39]:
title_index = title_book.index.values
title_index

array([3094], dtype=int64)

In [40]:
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes2 = genre_sim_sorted_ind2[title_index, :10]

print(similar_indexes)
print(similar_indexes2)

[[3094  192 1059  792 2821  267 2554 1552   36 2678]]
[[3094  192 1059  792 2821  267 2554 1552  587   36]]


## 연금술사 추천

### 기존 키워드 기반

In [41]:
similar_book = find_sim_book_ver1(df_CBF, genre_sim_sorted_ind, '연금술사', 10) # 책 제목 입력
similar_book[['title', 'score', 'total', 'keyword']]
# 문제 ; 평점 기반으로 추천하고자 하는데, vote_count가 낮은 책 제외하고 싶음

[[   1  653 2165 1432 1640  498 2946 1825  309 1234]]


,title,score,total,keyword
1,연금술사,9.6,350,마음 모험 연금술 만물 성장소설 영혼
653,작별하지 않는다,9.3,181,한국소설 삶 마음 사랑 작가 인선
2165,라이온의 간식,9.9,32,위로 인생 삶 마음 감동 소설
1432,필라테스 바이블(조셉 필라테스의),10.0,20,마음 로빈스 컨트롤 몸 트위스트 균형
1640,"그냥, 2200km를 걷다",8.8,4,모험 알베르 리스본 순례길 카미노 산티아고
498,정재승의 인간 탐구 보고서. 8: 불안이 온갖 미신을 만든다(어린이를 위한 뇌과학 ...,10.0,44,심리학 뇌과학 마음 외계인 지구인 루이
2946,귀멸의 칼날. 17,9.7,202,판타지만화 우정 성장 모험 시노부 상현
1825,귀멸의 칼날. 22,9.4,177,판타지만화 만화 성장 모험 카마 다이쇼
309,귀멸의 칼날. 12,9.8,217,판타지만화 우정 성장 모험 요리 상현
1234,귀멸의 칼날. 11,9.8,191,판타지만화 우정 성장 모험 도깨비 우즈


### 1.3.2 카테고리 추가된 키워드 기반

In [44]:
similar_book = find_sim_book_ver1(df_CBF, genre_sim_sorted_ind2, '연금술사', 10) # 책 제목 입력
similar_book[['title', 'score', 'total', 'keyword2']]
# 문제 ; 평점 기반으로 추천하고자 하는데, vote_count가 낮은 책은 제외하고 싶음

[[   1 2165 1013  767 3057 2351  653 2886 2021 2660]]


,title,score,total,keyword2
1,연금술사,9.6,350,소설 마음 모험 연금술 만물 성장소설 영혼
2165,라이온의 간식,9.9,32,소설 위로 인생 삶 마음 감동 소설
1013,호수의 일(양장본 HardCover),9.8,32,소설 마음 상처 호정 사춘기 아이 치유 겨울
767,그리스인 조르바(열린책들 세계문학 21)(양장본 HardCover),9.5,150,소설 세계고전문학 우정 성장 장편소설 소설 영혼
3057,쇼코의 미소,9.5,583,소설 소설집 마음 중 단편 소설 젊은작가상 타인 소설가
2351,수레바퀴 아래서(세계문학전집 50),9.5,90,소설 세계고전문학 노벨 문학상 자전 소설 신학교 성장소설 소년
653,작별하지 않는다,9.3,181,소설 한국소설 삶 마음 사랑 작가 인선
2886,불편한 편의점(15만부 기념 윈터 에디션),9.8,634,소설 한국소설 삶 위로 소설 독고 여사
2021,어린 왕자(양장본 HardCover),9.7,155,소설 세계고전문학 삶 소설 어린이 작품 번역
2660,자기 앞의 생(문학동네 세계문학)(양장본 HardCover),9.6,269,소설 소설 인생 삶 작가 사랑 모모


키워드에 카테고리(장르)를 추가한 것이 훨씬 추천 결과가 좋다. 
이는 기존의 교보문고의 키워드 선정이 매우 세부적이어서 개별 데이터(책) 사이의 공통된 키워드가 적기 때문이다. 
따라서 공통키워드로 카테고리를 추가하였다. 
그러나 리뷰의 갯수는 매우 작으나 평점이 높은 항목을 고려하기 위해 가중평점을 산출하여 컬럼으로 추가한다. 

### 가중폄점 반영

In [53]:
# 상위 50%에 해당하는 vote_count를 최소 투표 횟수인 m으로 지정
C = df_CBF['score'].mean()
m = df_CBF['total'].quantile(0.5)

In [54]:
def weighted_vote_average(record):
    v = record['total']
    R = record['score']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

In [55]:
# 가중평점(weighted_vote) 카테고리 추가
df_CBF['weighted_score'] = df_CBF.apply(weighted_vote_average, axis=1)
df_CBF.head(3)

,title,keyword,score,total,category,keyword2,weighted_score
0,첫 번째 디자인 케이크,디저트 초코 아이싱 플라워 제과제빵 밀크티,9.2,15,요리,요리 디저트 초코 아이싱 플라워 제과제빵 밀크티,9.399354
1,연금술사,마음 모험 연금술 만물 성장소설 영혼,9.6,350,소설,소설 마음 모험 연금술 만물 성장소설 영혼,9.599947
2,그때 그 마음(2022 제67회 현대문학상 수상소설집),순정 혜성 초파리 심사 폐허 허공,10.0,5,소설,소설 순정 혜성 초파리 심사 폐허 허공,9.699031


In [59]:
# 가중평점(weighted_vote) 기준으로 버터도 달걀도 필요 없는 건강빵과 유사한 책 추천
# 키워드2 기반
similar_movies = find_sim_book_ver2(df_CBF, genre_sim_sorted_ind2, '연금술사', 10)
similar_movies[['title', 'score', 'weighted_score', 'keyword2', 'total']]

,title,score,weighted_score,keyword2,total
2165,라이온의 간식,9.9,9.803843,소설 위로 인생 삶 마음 감동 소설,32
2886,불편한 편의점(15만부 기념 윈터 에디션),9.8,9.795348,소설 한국소설 삶 위로 소설 독고 여사,634
1013,호수의 일(양장본 HardCover),9.8,9.735758,소설 마음 상처 호정 사춘기 아이 치유 겨울,32
2021,어린 왕자(양장본 HardCover),9.7,9.691062,소설 세계고전문학 삶 소설 어린이 작품 번역,155
2660,자기 앞의 생(문학동네 세계문학)(양장본 HardCover),9.6,9.599932,소설 소설 인생 삶 작가 사랑 모모,269
2351,수레바퀴 아래서(세계문학전집 50),9.5,9.514101,소설 세계고전문학 노벨 문학상 자전 소설 신학교 성장소설 소년,90
767,그리스인 조르바(열린책들 세계문학 21)(양장본 HardCover),9.5,9.508973,소설 세계고전문학 우정 성장 장편소설 소설 영혼,150
3057,쇼코의 미소,9.5,9.502476,소설 소설집 마음 중 단편 소설 젊은작가상 타인 소설가,583
653,작별하지 않는다,9.3,9.322860,소설 한국소설 삶 마음 사랑 작가 인선,181


In [67]:
# 가중평점(weighted_vote) 기준으로 버터도 달걀도 필요 없는 건강빵과 유사한 책 추천
# 키워드2 기반
similar_movies = find_sim_book_ver2(df_CBF, genre_sim_sorted_ind2, '원숭이도 이해하는 자본론(새로 쓴)', 10)
similar_movies[['title', 'score', 'weighted_score', 'keyword2', 'total']]

,title,score,weighted_score,keyword2,total
1572,구별짓기 (상)(21세기총서 3),9.9,9.703505,정치사회 취향 자본 비판 실천 경제 소비,8
2776,진정성이라는 거짓말,9.7,9.639225,정치사회 철학자 민낯 사회비평 아우라 자평 대량,10
825,21세기 자본(양장본 HardCover),9.6,9.599596,정치사회 사회학 불평등 평등 소득 격차 분배 자본론,33
1805,이재명의 나의 소년공 다이어리,9.5,9.567301,정치사회 일기 성남 기록 공장 일기장 시절,7
162,올바름이라는 착각,9.2,9.286676,정치사회 개인 혐오 정치 페미니즘 피터슨 조던,54
307,한국의 능력주의,8.1,9.224031,정치사회 평등 찬성 공정 정규직 공채 한국인,5
846,지속 불가능 자본주의,8.4,9.181766,정치사회 철학자 열중 마르크스 환경보호 기후 변화 인류세,8
2893,새로운 가난이 온다,8.7,9.085161,정치사회 혐오 노동 자본주의 노동자 생존 산업 혁명,20
115,눈 떠보니 선진국,8.5,8.749706,정치사회 사회학 볼륨 바이든 정책 한국 사회 경제발전,51
